## Data Load
- All Vends Data
- Machine Last and First Dates

In [ ]:
import pandas as pd
import numpy as np
# import datetime as dt

In [ ]:
df= pd.read_csv('/content/DFORE1650909.csv', low_memory=False)
mf= pd.read_csv('/content/DFORE165mcminmaxdt.csv')

In [ ]:
df.columns

Index(['cohort_flag', 'location_id', 'as400id', 'accountname', 'branchname',
       'propertyclass', 'age_group', 'machine_rogue_flag', 'machine_id',
       'machine_type', 'installcap', 'city', 'transactor_id', 'zipcode',
       'latitude', 'longitude', 'payment_type', 'use_type', 'transaction_date',
       'first_transaction_date', 'no_of_vends', 'collected_amount',
       'vend_amount', 'transaction_fee_amount', 'PrcntCont', 'TXN_SPAM_FLG'],
      dtype='object')

In [ ]:
len(df.location_id.unique())

3758

## Installed Capacity

In [ ]:
df.groupby('location_id').agg({'installcap': 'max', 'machine_id': 'nunique'}).iloc[:,0]\
[df.groupby('location_id').agg({'installcap': 'max', 'machine_id': 'nunique'}).iloc[:,1]/\
df.groupby('location_id').agg({'installcap': 'max', 'machine_id': 'nunique'}).iloc[:,0]<= .85]

location_id
00057aa7-d19b-437d-856d-e92f7c92112d    14
00b1f906-9d5b-40ee-9546-1e852ecff2f1    30
00cac1ef-f051-4977-a1be-1f03e4c1f581    10
01272c71-39ba-4d97-81a0-eb6e1fe00d4c     6
0137d235-818e-48d0-8a7e-d382f04def84     4
                                        ..
fe4c788d-34ba-4575-bd45-207005bf2f9c     4
ffbbd0db-bddb-4447-9a68-e5c975be3019     5
ffd602d2-41b9-4aba-a92f-c1338df270cc    10
ffec73b0-42bb-406f-89a4-f8a2b4961e52    20
fff7af50-bec9-4da1-8d39-4dc70049111f     8
Name: installcap, Length: 1027, dtype: int64

In [ ]:
df.groupby('machine_id')['installcap'].nunique()[df.groupby('machine_id')['installcap'].nunique()> 1]

machine_id
454-BZP    2
455-BZP    2
457-BZP    2
637-GRT    2
Name: installcap, dtype: int64

## Unused Machine Count

In [ ]:
len(df.machine_id.unique())

28332

In [ ]:
mf.shape[0]- len(df.machine_id.unique())

9408

## Features

In [ ]:
df= df.loc[df.TXN_SPAM_FLG== 'HAM']

In [ ]:
df= df.loc[df.machine_rogue_flag=='N']

In [ ]:
df.shape

(613103, 26)

In [ ]:
df.columns

Index(['cohort_flag', 'location_id', 'as400id', 'accountname', 'branchname',
       'propertyclass', 'age_group', 'machine_rogue_flag', 'machine_id',
       'machine_type', 'installcap', 'city', 'transactor_id', 'zipcode',
       'latitude', 'longitude', 'payment_type', 'use_type', 'transaction_date',
       'first_transaction_date', 'no_of_vends', 'collected_amount',
       'vend_amount', 'transaction_fee_amount', 'PrcntCont', 'TXN_SPAM_FLG'],
      dtype='object')

In [ ]:
md= df.groupby('machine_id').agg(DaysWVends=('transaction_date', 'nunique')).reset_index()

In [ ]:
df1= df.merge(mf, how='inner', left_on='machine_id', right_on='machine_id')

In [ ]:
df1= df1.merge(md,  how='inner', left_on='machine_id', right_on='machine_id')

In [ ]:
df1.columns

Index(['cohort_flag', 'location_id', 'as400id', 'accountname', 'branchname',
       'propertyclass', 'age_group', 'machine_rogue_flag', 'machine_id',
       'machine_type', 'installcap', 'city', 'transactor_id', 'zipcode',
       'latitude', 'longitude', 'payment_type', 'use_type', 'transaction_date',
       'first_transaction_date', 'no_of_vends', 'collected_amount',
       'vend_amount', 'transaction_fee_amount', 'PrcntCont', 'TXN_SPAM_FLG',
       'MinTxnDt', 'MaxTxnDt', 'DaysWVends'],
      dtype='object')

In [ ]:
# pctl_series=df.no_of_vends.describe(percentiles=pcts)
df1['AppUser']= df1.loc[df1.payment_type=='Stored-Value', 'transactor_id']
df1['AppSpend']= df1.loc[df1.payment_type=='Stored-Value','collected_amount']
df1['AppVend']= df1.loc[df1.payment_type== 'Stored-Value', 'no_of_vends']
df1['DigitalUser']= df1.loc[df1.payment_type!='Coin', 'transactor_id']
df1['DigitalSpend']= df1.loc[df1.payment_type!='Coin','collected_amount']
df1['DigitalVend']= df1.loc[df1.payment_type!='Coin', 'no_of_vends']

In [ ]:
df1['MachineinPeriod']= pd.to_datetime(np.where(df1.MinTxnDt <= '2021-08-04', '2021-08-04 00:00:00', df1.MinTxnDt))

In [ ]:
df1['MachineDaysinPeriod']= round((pd.to_datetime('2021-09-09')- df1.MachineinPeriod)/pd.offsets.Day(1),0)

In [ ]:
df1['Tenured']= df1.MachineDaysinPeriod >= 30

In [ ]:
df1.groupby('machine_id')['MachineDaysinPeriod'].nunique()[df1.groupby('machine_id')['MachineDaysinPeriod'].nunique()> 1]

Series([], Name: MachineDaysinPeriod, dtype: int64)

In [ ]:
len(df1.location_id.unique())

3755

In [ ]:
df1=df1.loc[~df1.DigitalUser.isnull()]

In [ ]:
df1.isnull().sum()

cohort_flag                    0
location_id                    0
as400id                        0
accountname                    0
branchname                     0
propertyclass                  0
age_group                      0
machine_rogue_flag             0
machine_id                     0
machine_type                 282
installcap                     0
city                           0
transactor_id                  0
zipcode                        0
latitude                       0
longitude                      0
payment_type                   0
use_type                       0
transaction_date               0
first_transaction_date         0
no_of_vends                    0
collected_amount               0
vend_amount                    0
transaction_fee_amount         0
PrcntCont                      0
TXN_SPAM_FLG                   0
MinTxnDt                       0
MaxTxnDt                       0
DaysWVends                     0
AppUser                   102694
AppSpend  

In [189]:
df1.transaction_date= pd.to_datetime(df1.transaction_date)

In [190]:
df1.transaction_date.dt.week

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """Entry point for launching an IPython kernel.


0         34
1         34
2         33
3         34
4         31
          ..
613018    36
613069    35
613072    32
613073    32
613074    32
Name: transaction_date, Length: 265794, dtype: int64

## Location Level

In [ ]:
def location_mets(x):
  names= {
      'locations': x['location_id'].nunique(),
      'InstalledMachines': x['installcap'].max(),
      'TxnedMachines': x['machine_id'].nunique(),
      'DaysWVend': x['transaction_date'].nunique(),
      'AllVends': x['no_of_vends'].sum(),
      'AllVendAmount': x['collected_amount'].sum()/100.00,
      'AppUsers': x['AppUser'].nunique(), 
      'AppSpends': x['AppSpend'].sum()/100.00, 
      'AppVends': x['AppVend'].sum(),
      'DigitalUsers': x['DigitalUser'].nunique(), 
      'DigitalSpends': x['DigitalSpend'].sum()/100.00, 
      'DigitalVends': x['DigitalVend'].sum(),
      'GRPM': 30.42*x['collected_amount'].sum()/(x['transaction_date'].nunique()*x['installcap'].max()*100.00),
      'RPTM': 30.42*x['collected_amount'].sum()/(x['transaction_date'].nunique()*x['machine_id'].nunique()*100.00),
      'ARPAU':30.42*x['AppSpend'].sum()/(x['transaction_date'].nunique()*x['AppUser'].nunique()*100.00),
      'VendsDigitalUser': 30.42*x['DigitalVend'].sum()/(x['transaction_date'].nunique()*x['DigitalUser'].nunique())

  }
  return(pd.Series(names, index=list(names.keys())))

In [ ]:
df2=df1.groupby(['cohort_flag', 'location_id']).apply(location_mets).reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
def cohort_mets(x):
  names= {
      'locations': x['locations'].sum(),
      'InstalledMachines': x['InstalledMachines'].sum(),
      'TxnedMachines': x['TxnedMachines'].sum(),
      #'DaysWVend': x['DaysWVend'].sum(),
      'AllVends': x['AllVends'].sum(),
      'AllVendAmount': x['AllVendAmount'].sum(),
      'AppUsers': x['AppUsers'].sum(), 
      'AppSpends': x['AppSpends'].sum(), 
      'AppVends': x['AppVends'].sum(),
      'DigitalUsers': x['DigitalUsers'].sum(), 
      'DigitalSpends': x['DigitalSpends'].sum(), 
      'DigitalVends': x['DigitalVends'].sum(),
      'GRPM': x['GRPM'].mean(),
      'RPTM': x['RPTM'].mean(),
      'ARPAU':x['ARPAU'].mean(),
      'VendsDigitalUser': x['VendsDigitalUser'].mean()

  }
  return(pd.Series(names, index=list(names.keys())))

In [ ]:
df2.describe()

,locations,InstalledMachines,TxnedMachines,DaysWVend,AllVends,AllVendAmount,AppUsers,AppSpends,AppVends,DigitalUsers,DigitalSpends,DigitalVends,GRPM,RPTM,ARPAU,VendsDigitalUser
count,3148.0,3148.000000,3148.000000,3148.000000,3148.000000,3148.000000,3148.000000,3148.000000,3148.000000,3148.000000,3148.000000,3148.000000,3148.000000,3148.000000,2793.000000,3148.000000
mean,1.0,9.340534,7.102922,13.094346,100.260165,165.202516,6.273825,101.613612,62.484435,12.070203,165.202516,100.260165,37.423113,45.827076,51.049574,32.206327
std,0.0,9.054700,7.133407,10.604287,259.198417,431.200113,14.130424,259.245539,159.447884,26.824948,431.200113,259.198417,29.194782,29.271947,63.534622,40.573618
min,1.0,1.000000,1.000000,1.000000,1.000000,0.750000,0.000000,0.000000,0.000000,1.000000,0.750000,1.000000,1.358036,3.802500,2.401579,3.105375
25%,1.0,4.000000,3.000000,4.000000,12.000000,18.000000,1.000000,8.250000,5.000000,2.000000,18.000000,12.000000,17.111250,25.408792,19.137857,11.598599
50%,1.0,7.000000,5.000000,10.000000,38.000000,62.500000,3.000000,36.750000,23.000000,6.000000,62.500000,38.000000,28.953676,38.975625,31.239000,19.785071
75%,1.0,12.000000,8.000000,20.000000,101.000000,168.687500,7.000000,103.562500,63.000000,12.000000,168.687500,101.000000,48.941688,57.037500,58.938750,35.490000
max,1.0,186.000000,118.000000,37.000000,7923.000000,14023.250000,365.000000,7488.500000,4240.000000,693.000000,14023.250000,7923.000000,285.187500,296.595000,1095.120000,730.080000


In [ ]:
df2.groupby(['cohort_flag']).apply(cohort_mets)

,locations,InstalledMachines,TxnedMachines,AllVends,AllVendAmount,AppUsers,AppSpends,AppVends,DigitalUsers,DigitalSpends,DigitalVends,GRPM,RPTM,ARPAU,VendsDigitalUser
cohort_flag,,,,,,,,,,,,,,,
control,588.0,5581.0,4168.0,51064.0,82050.50,3362.0,52314.11,32897.0,6346.0,82050.50,51064.0,36.821533,45.593013,51.842308,33.178146
test,2560.0,23823.0,18192.0,264555.0,438007.02,16388.0,267565.54,163804.0,31651.0,438007.02,264555.0,37.561289,45.880837,50.871634,31.983113


In [ ]:
df1.groupby(['cohort_flag']).apply(location_mets)

,locations,InstalledMachines,TxnedMachines,DaysWVend,AllVends,AllVendAmount,AppUsers,AppSpends,AppVends,DigitalUsers,DigitalSpends,DigitalVends,GRPM,RPTM,ARPAU,VendsDigitalUser
cohort_flag,,,,,,,,,,,,,,,,
control,710.0,88.0,5299.0,37.0,182119.0,241922.52,3322.0,52315.86,32898.0,6306.0,82759.77,51583.0,2260.222070,37.535298,12.947658,6.725276
test,3045.0,186.0,22969.0,37.0,866624.0,1186046.51,16016.0,267569.04,163806.0,31279.0,442278.53,267541.0,5242.594425,42.453854,13.735336,7.032261


In [ ]:
df2.head(2).T

,0,1
cohort_flag,control,control
location_id,1699d62c-7c78-4b44-af1e-1ef4193f79bb,4470090
locations,1,1
InstalledMachines,6,6
TxnedMachines,1,1
DaysWVend,2,25
AllVends,2,45
AllVendAmount,3,67.5
AppUsers,2,8
AppSpends,3,67.5


In [ ]:
df3=df1.groupby(['cohort_flag']).apply(location_mets).reset_index()

In [ ]:
df3.head(2).T

,0,1
cohort_flag,control,test
locations,3,16
InstalledMachines,7,48
TxnedMachines,4,55
DaysWVend,28,37
AllVends,49,578
AllVendAmount,73.25,991.75
AppUsers,11,85
AppSpends,73.25,485.25
AppVends,49,273


In [ ]:
df2.groupby('cohort_flag')['GRPM',	'RPTM',	'ARPAU',	'VendsDigitalUser'].agg('mean')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,GRPM,RPTM,ARPAU,VendsDigitalUser
cohort_flag,,,,
control,11.081571,56.530500,38.912250,27.631500
test,17.438409,62.698696,22.786407,17.085426


In [ ]:
met_dict= {'vends':{'no_of_vends': 'sum'}, 
           'locations':{'location_id': 'nunique'},
           'machine_id':'nunique', 
           'transaction_date': 'nunique',
           'collected_amount': 'sum', 
           'transactor_id': 'nunique',
           'AppUser': 'nunique', 
           'AppSpend': 'sum', 
           'AppVend': 'sum',
           'DigitalUser': 'nunique', 
           'DigitalSpend': 'sum',
           'DigitalVend': 'sum',
           'DaysinPeriod': 'sum'}

In [ ]:
df3=df2.groupby('cohort_flag').agg(met_dict)

In [ ]:
df3

,no_of_vends,location_id,machine_id,collected_amount,transactor_id,AppUser,AppSpend,DigitalUser,DigitalSpend,DigitalVend,DaysinPeriod,ARPAU,GRPM,VendsDigitalUser,SpendDigitalUser,RPMD
cohort_flag,,,,,,,,,,,,,,,,
control,143189,664,4795,18957799,5252,2781,4047418.0,5252,6401269.0,32856,2437737.0,14.553822,39.536599,6.255903,12.188250,7.776802
test,683519,2869,21551,93466440,26815,13781,20755016.0,26815,34567413.0,176684,12028286.0,15.060602,43.369885,6.588999,12.891073,7.770554


In [ ]:
# df3['UsedMachinePct']= df3.TxnMachines/df3.InstalledMachines
df3['ARPAU']= df3.AppSpend/(100.00*df3.AppUser)
df3['GRPM']= df3.collected_amount/(100.0*df3.machine_id)
df3['VendsDigitalUser']= df3.DigitalVend/df3.DigitalUser
df3['SpendDigitalUser']= df3.DigitalSpend/ (100.0*df3.DigitalUser)
df3['RPMD']= df3.collected_amount/ df3.DaysinPeriod


In [ ]:
df2.transaction_date.dt

AttributeError: ignored

In [ ]:
df4= df2.groupby(['cohort_flag', 'Tenured', ]).agg(met_dict)

In [ ]:
df4['ARPAU']= df4.AppSpend/(100.00*df4.AppUser)
df4['GRPM']= df4.collected_amount/(100.0*df4.machine_id)
df4['VendsDigitalUser']= df4.DigitalVend/df4.DigitalUser
df4['SpendDigitalUser']= df4.DigitalSpend/ (100.0*df4.DigitalUser)
df4['RPMD']= df4.collected_amount/ df4.DaysinPeriod

In [ ]:
df4

no_of_vends  location_id  ...  SpendDigitalUser       RPMD
cohort_flag Tenured                            ...                             
control     False          15420          250  ...          6.836959  10.416202
            True          127769          491  ...         13.076882   7.535411
test        False          65368         1040  ...          7.115000  11.912175
            True          618151         2187  ...         13.510516   7.483929

[4 rows x 16 columns]

In [ ]:
pcts=[.05,.1,.2,.25,.5,.75,.9,.95,.96,.97,.98,.99,.995,.996,.997,.998,.999]
pctls=['5%', '10%', '20%', '25%', '50%', '75%', '90%', '95%', '96%', '97%', '98%', '99%', '99.5%', '99.6%', '99.7%', '99.8%', '99.9%']
pctl_series=df.no_of_vends.describe(percentiles=pcts)
df['AppUser']= df.loc[df.payment_type=='Stored-Value', 'transactor_id']
df['AppSpend']= df.loc[df.payment_type=='Stored-Value','collected_amount']
df['DigitalUser']= df.loc[df.payment_type!='Coin', 'transactor_id']
df['DigitalSpend']= df.loc[df.payment_type!='Coin','collected_amount']
df['DigitalVend']= df.payment_type!='Coin' 
met_dict= {'installcap': 'min', 
           'no_of_vends': 'sum', 
           'machine_id':'nunique',
           'collected_amount': 'sum', 
           'transactor_id': 'nunique',
           'AppUser': 'nunique', 
           'AppSpend': 'sum', 
           'DigitalUser': 'nunique', 
           'DigitalSpend': 'sum',
           'DigitalVend': 'sum'}

In [ ]:
v= pctl_series[pctls[-2]]
df1= df.loc[df.no_of_vends <= v]

In [ ]:
df2= df1.groupby(['cohort_flag', 'location_id']).agg(met_dict).reset_index()
df2.columns=['cohort_flag', 'location_id','InstalledMachines', 'Vends', 'TxnMachines', 'SpendAmount', "AllUser", 
             'AppUser', 'AppSpendAmount', 'DigitalUser','DigitalSpend', 'DigitalVend']

In [ ]:
df2.head()

,cohort_flag,location_id,InstalledMachines,Vends,TxnMachines,SpendAmount,AllUser,AppUser,AppSpendAmount,DigitalUser,DigitalSpend,DigitalVend,UsedMachinePct,ARPAU,GRPM,VendsDigitalUser,SpendDigitalUser
0,control,001c980e-9557-4f85-8336-7653d85a618b,15,194,10,21875,1,1,650.0,1,650.0,5,0.666667,6.500,21.875000,5.000000,650.000000
1,control,01272c71-39ba-4d97-81a0-eb6e1fe00d4c,5,2,2,375,1,1,175.0,1,175.0,1,0.400000,1.750,1.875000,1.000000,175.000000
2,control,01ce7901-de7f-4ba8-835c-4a852455678e,11,66,9,10700,14,4,1950.0,14,7800.0,46,0.818182,4.875,11.888889,3.285714,557.142857
3,control,026926d1-e09a-42b1-b231-ff264495a10a,2,6,2,650,2,0,0.0,2,650.0,4,1.000000,NaN,3.250000,2.000000,325.000000
4,control,02e08450-d3f5-4e57-8487-96ec9cc64546,4,15,3,2250,0,0,0.0,0,0.0,0,0.750000,NaN,7.500000,NaN,NaN


In [ ]:
df2['UsedMachinePct']= df2.TxnMachines/df2.InstalledMachines
df2['ARPAU']= df2.AppSpendAmount/(100.00*df2.AppUser)
df2['GRPM']= df2.SpendAmount/(100.0*df2.TxnMachines)
df2['VendsDigitalUser']= df2.DigitalVend/df2.DigitalUser
df2['SpendDigitalUser']= df2.DigitalSpend/ (100.0*df2.DigitalUser)

In [ ]:
df3= df2.groupby('cohort_flag').agg({'location_id': 'nunique','InstalledMachines': 'sum','Vends': 'sum',
       'TxnMachines': 'sum', 'SpendAmount': 'sum', 'AllUser': 'sum', 'AppUser': 'sum', 'AppSpendAmount': 'sum',
       'DigitalUser': 'sum', 'DigitalSpend': 'sum', 'DigitalVend': 'sum' })

In [ ]:
df2.head()

,cohort_flag,location_id,InstalledMachines,Vends,TxnMachines,SpendAmount,AllUser,AppUser,AppSpendAmount,DigitalUser,DigitalSpend,DigitalVend,UsedMachinePct,ARPAU,GRPM,VendsDigitalUser,SpendDigitalUser
0,control,001c980e-9557-4f85-8336-7653d85a618b,15,194,10,21875,1,1,650.0,1,650.0,5,0.666667,6.500,21.875000,5.000000,6.500000
1,control,01272c71-39ba-4d97-81a0-eb6e1fe00d4c,5,2,2,375,1,1,175.0,1,175.0,1,0.400000,1.750,1.875000,1.000000,1.750000
2,control,01ce7901-de7f-4ba8-835c-4a852455678e,11,66,9,10700,14,4,1950.0,14,7800.0,46,0.818182,4.875,11.888889,3.285714,5.571429
3,control,026926d1-e09a-42b1-b231-ff264495a10a,2,6,2,650,2,0,0.0,2,650.0,4,1.000000,NaN,3.250000,2.000000,3.250000
4,control,02e08450-d3f5-4e57-8487-96ec9cc64546,4,15,3,2250,0,0,0.0,0,0.0,0,0.750000,NaN,7.500000,NaN,NaN


In [ ]:
df3['UsedMachinePct']= df3.TxnMachines/df3.InstalledMachines
df3['ARPAU']= df3.AppSpendAmount/(100.00*df3.AppUser)
df3['GRPM']= df3.SpendAmount/(100.0*df3.TxnMachines)
df3['VendsDigitalUser']= df3.DigitalVend/df3.DigitalUser
df3['SpendDigitalUser']= df3.DigitalSpend/ (100.0*df3.DigitalUser)
df3['VendsLocation']= df3.Vends/df3.location_id
df3['SpendLocation']= df3.DigitalSpend/(100.00*df3.location_id)

In [ ]:
df_mean= df3.loc[:, ['ARPAU', 'GRPM',
       'VendsDigitalUser', 'SpendDigitalUser', 'VendsLocation',
       'SpendLocation']].T

cohort_flag,control,test
ARPAU,14.200902,14.433839
GRPM,38.173090,41.435779
VendsDigitalUser,6.119561,6.379515
SpendDigitalUser,11.963317,12.491835
VendsLocation,207.752705,225.959350
SpendLocation,92.637125,113.941078


In [ ]:
from scipy import stats
for c in ['ARPAU', 'GRPM', 'VendsDigitalUser', 'SpendDigitalUser', 'DigitalVend', 'DigitalSpend']:
  control= df2.loc[(df2.cohort_flag== 'control'), c]
  test= df2.loc[(df2.cohort_flag== 'test'), c]
  t_score = stats.ttest_ind_from_stats(test.mean(), test.std(), test.shape[0], \
                               control.mean(), control.std(), control.shape[0], \
                               equal_var=False)
  print(c, test.mean(), test.std(), control.mean(), control.std(),t_score)



ARPAU 13.120125402778847 9.16162273705943 13.714254903980938 10.630355178929287 Ttest_indResult(statistic=-1.3143662446891735, pvalue=0.18906610852572037)
GRPM 37.57000023106597 31.821648774649788 36.097299496655815 32.15147501837784 Ttest_indResult(statistic=1.0530983055281504, pvalue=0.292561781066832)
VendsDigitalUser inf nan 5.568468651933381 3.6778008340779937 Ttest_indResult(statistic=nan, pvalue=nan)
SpendDigitalUser inf nan 11.42951963373264 8.377566015593194 Ttest_indResult(statistic=nan, pvalue=nan)
DigitalVend 58.18911276069282 179.19688608696444 47.38639876352396 107.54314707804788 Ttest_indResult(statistic=1.9982266641918858, pvalue=0.04586307787416769)
DigitalSpend 11394.107811947684 33657.50530603499 9263.712519319939 19538.750292074703 Ttest_indResult(statistic=2.140603992657296, pvalue=0.03245245108999421)


In [ ]:
df2.loc[df2['UsedMachinePct']<=df2.UsedMachinePct.describe(percentiles=[.05,.1,.15,.25, .5,.75, .95])['10%']]

,cohort_flag,location_id,InstalledMachines,Vends,TxnMachines,SpendAmount,DigitalUser,AppUser,AppSpendAmount,UsedMachinePct
9,control,06843e7c-e73d-4c91-b5c8-74a793538dc4,46,1,1,25,0,0,0.0,0.021739
13,control,0719ddde-2727-400d-92ee-251306a0d8bb,64,346,15,48550,4,3,4000.0,0.234375
14,control,0743f6ab-de61-456e-972f-e1d0092eb5ec,36,574,8,56075,17,8,11050.0,0.222222
22,control,0aae2157-f369-426e-90c0-755f8565e5f4,64,21,7,3150,1,0,0.0,0.109375
24,control,0afef6a8-e558-4392-8418-3d081c284eea,343,88,26,13150,4,1,225.0,0.075802
...,...,...,...,...,...,...,...,...,...,...
3454,test,fd1d810f-6623-43ab-83fe-6fb26500e2c4,19,6,3,1200,2,1,400.0,0.157895
3460,test,fdf45964-7fb1-4cb6-b8d9-0f35e8c62663,162,977,24,131000,17,9,14900.0,0.148148
3471,test,ff7e4613-5c05-4a30-8f40-8220324a4cd3,100,750,20,121425,17,15,23125.0,0.200000
3473,test,ffbbd0db-bddb-4447-9a68-e5c975be3019,5,1,1,25,0,0,0.0,0.200000


# 2 months GRPM

In [ ]:
import pandas as pd
gf= pd.read_csv('/content/TwoMnthsGRPM.csv')

In [ ]:
gf.groupby('MonthYear')['rev', 'machines'].sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,rev,machines
MonthYear,,
2021-07,2086161.00,26830
2021-08,2943900.24,36832


In [ ]:
2086161.00/ 26830

77.75478941483414

In [ ]:
2943900.24/ 36832

79.92778670721113